In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════
VGG16 Baseline - Dog vs Cat Classifier
═══════════════════════════════════════════════════════════════════════════

Binary image classification using transfer learning with VGG16.

Features:
- Transfer learning from ImageNet weights
- Data augmentation (flip + brightness)
- Early stopping & model checkpointing
- Comprehensive evaluation metrics
- Confusion matrix & prediction visualization

Performance: ~93-94% validation accuracy

Author: Mohamed Sherif Ali
"""

═══════════════════════════════════════════════════════════════════════════
STEP 1: Import Libraries
═══════════════════════════════════════════════════════════════════════════

In [ ]:
import os
import random
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    CSVLogger,
    TensorBoard
)

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_curve,
    auc as sklearn_auc
)

In [ ]:
print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

═══════════════════════════════════════════════════════════════════════════
STEP 2: Configuration & Setup
═══════════════════════════════════════════════════════════════════════════

In [ ]:
# Random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 50  # Will stop early with callbacks
LEARNING_RATE = 1e-4
IMAGE_SIZE = (224, 224)

In [ ]:
# Create output directories
os.makedirs('models', exist_ok=True)
os.makedirs('plots', exist_ok=True)
os.makedirs('logs', exist_ok=True)

In [ ]:
print(f"\n{'='*60}")
print(f"  VGG16 BASELINE CONFIGURATION")
print(f"{'='*60}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Image Size: {IMAGE_SIZE}")
print(f"Random Seed: {SEED}")
print(f"{'='*60}\n")

═══════════════════════════════════════════════════════════════════════════
STEP 3: GPU Configuration
═══════════════════════════════════════════════════════════════════════════

In [ ]:
# Configure GPU memory growth (prevents OOM errors)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ {len(gpus)} GPU(s) configured with memory growth")
        print(f"   GPUs: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(f"⚠️  GPU configuration error: {e}")
else:
    print("ℹ️  No GPU detected, using CPU")

═══════════════════════════════════════════════════════════════════════════
STEP 4: Load Dataset
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n📥 Loading Cats vs Dogs dataset...")

In [ ]:
# Load with seed for reproducibility
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    read_config=tfds.ReadConfig(shuffle_seed=SEED)
)

In [ ]:
# Display dataset info
print("\n📊 Dataset Information:")
print(f"Total examples: {ds_info.splits['train'].num_examples}")
print(f"Training examples: {int(ds_info.splits['train'].num_examples * 0.8)}")
print(f"Validation examples: {int(ds_info.splits['train'].num_examples * 0.2)}")
print(f"Classes: {ds_info.features['label'].names}")
print(f"Features: {ds_info.features}")

In [ ]:
# Verify a sample
print("\n🔍 Verifying data sample...")
for image, label in train_ds.take(1):
    print(f"Image shape: {image.shape}")
    print(f"Image dtype: {image.dtype}")
    print(f"Label: {label.numpy()} ({ds_info.features['label'].names[label.numpy()]})")
    assert image.shape[2] == 3, "Expected RGB images"
    assert label.numpy() in [0, 1], "Expected binary labels"
print("✅ Data verification passed!")

═══════════════════════════════════════════════════════════════════════════
STEP 5: Data Preprocessing
═══════════════════════════════════════════════════════════════════════════

In [ ]:
def preprocess(image, label):
    """
    Resize and normalize images
    
    Args:
        image: Input image tensor
        label: Class label (0=cat, 1=dog)
    
    Returns:
        Preprocessed image and label
    """
    # Resize to 224x224 (VGG16 input size)
    image = tf.image.resize(image, IMAGE_SIZE)
    
    # Normalize to [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    
    return image, label

In [ ]:
def augment(image, label):
    """
    Apply data augmentation to training images
    
    Augmentations:
    - Random horizontal flip (50% chance)
    - Random brightness adjustment (±10%)
    
    Args:
        image: Preprocessed image
        label: Class label
    
    Returns:
        Augmented image and label
    """
    # Random horizontal flip
    image = tf.image.random_flip_left_right(image)
    
    # Random brightness adjustment
    image = tf.image.random_brightness(image, max_delta=0.1)
    
    # Clip values to [0, 1] range
    image = tf.clip_by_value(image, 0.0, 1.0)
    
    return image, label

In [ ]:
print("\n✅ Preprocessing functions defined")

═══════════════════════════════════════════════════════════════════════════
STEP 6: Create Data Pipeline
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n🔄 Building optimized data pipeline...")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Training pipeline with caching and prefetching
train_batches = (train_ds
    .cache()  # Cache preprocessed data in memory
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .shuffle(buffer_size=10000, seed=SEED)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE))

In [ ]:
# Validation pipeline (no augmentation)
val_batches = (val_ds
    .cache()
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE))

In [ ]:
# Count batches
train_batch_count = tf.data.experimental.cardinality(train_batches).numpy()
val_batch_count = tf.data.experimental.cardinality(val_batches).numpy()

In [ ]:
print(f"✅ Training batches: {train_batch_count}")
print(f"✅ Validation batches: {val_batch_count}")

═══════════════════════════════════════════════════════════════════════════
STEP 7: Build VGG16 Model
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n🏗️  Building VGG16 model...")

In [ ]:
# Load pre-trained VGG16 (without top classification layer)
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [ ]:
# Freeze base model weights (transfer learning)
base_model.trainable = False

In [ ]:
print(f"✅ VGG16 base loaded with {len(base_model.layers)} layers (frozen)")

In [ ]:
# Build classifier on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(name='global_avg_pool'),
    Dense(256, activation='relu', name='dense_256'),
    Dropout(0.5, name='dropout_0.5'),
    Dense(1, activation='sigmoid', name='output')
], name='VGG16_Classifier')

In [ ]:
# Display model summary
print("\n" + "="*60)
print("MODEL ARCHITECTURE")
print("="*60)
model.summary()
print("="*60)

═══════════════════════════════════════════════════════════════════════════
STEP 8: Compile Model
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n⚙️  Compiling model...")

In [ ]:
# Custom optimizer with specific learning rate
optimizer = Adam(learning_rate=LEARNING_RATE)

In [ ]:
# Compile with multiple metrics
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        AUC(name='auc'),
        Precision(name='precision'),
        Recall(name='recall')
    ]
)

In [ ]:
print("✅ Model compiled with:")
print(f"   Optimizer: Adam (lr={LEARNING_RATE})")
print(f"   Loss: Binary Crossentropy")
print(f"   Metrics: Accuracy, AUC, Precision, Recall")

═══════════════════════════════════════════════════════════════════════════
STEP 9: Setup Callbacks
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n📞 Setting up training callbacks...")

In [ ]:
# Timestamp for file versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# Define callbacks
callbacks = [
    # Early stopping - stop when validation loss stops improving
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model checkpoint - save best model
    ModelCheckpoint(
        filepath=f'models/vgg16_best_{timestamp}_{{epoch:02d}}_{{val_accuracy:.4f}}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    ),
    
    # CSV logger - save training history
    CSVLogger(
        filename=f'logs/training_log_vgg16_{timestamp}.csv',
        append=False
    ),
    
    # TensorBoard - visualize training
    TensorBoard(
        log_dir=f'logs/tensorboard_vgg16_{timestamp}',
        histogram_freq=1
    )
]

In [ ]:
print("✅ Callbacks configured:")
print("   • Early Stopping (patience=5)")
print("   • Model Checkpoint (save best)")
print("   • Learning Rate Reduction (factor=0.5)")
print("   • CSV Logger")
print("   • TensorBoard")

═══════════════════════════════════════════════════════════════════════════
STEP 10: Train Model
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n" + "="*60)
print("  STARTING TRAINING")
print("="*60)
print(f"Max Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Training Batches: {train_batch_count}")
print(f"Validation Batches: {val_batch_count}")
print("="*60 + "\n")

In [ ]:
# Train the model
history = model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
print("\n" + "="*60)
print("  TRAINING COMPLETED")
print("="*60)
print(f"Epochs trained: {len(history.history['accuracy'])}")
print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
print("="*60 + "\n")

═══════════════════════════════════════════════════════════════════════════
STEP 11: Plot Training History
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("📊 Generating training plots...")

In [ ]:
# Extract history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

In [ ]:
# Accuracy plot
axes[0].plot(epochs_range, acc, 'b-', label='Training Accuracy', linewidth=2)
axes[0].plot(epochs_range, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
axes[0].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

In [ ]:
# Loss plot
axes[1].plot(epochs_range, loss, 'b-', label='Training Loss', linewidth=2)
axes[1].plot(epochs_range, val_loss, 'r-', label='Validation Loss', linewidth=2)
axes[1].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

In [ ]:
plt.tight_layout()
plt.savefig(f'plots/vgg16_training_curves_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print(f"✅ Training curves saved to: plots/vgg16_training_curves_{timestamp}.png")

═══════════════════════════════════════════════════════════════════════════
STEP 12: Evaluate Model
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n" + "="*60)
print("  MODEL EVALUATION")
print("="*60)

In [ ]:
# Evaluate on validation set
print("\n🔍 Evaluating on validation set...")
results = model.evaluate(val_batches, verbose=1)

In [ ]:
print("\n📊 Final Metrics:")
for metric_name, value in zip(model.metrics_names, results):
    print(f"   {metric_name.capitalize()}: {value:.4f}")

═══════════════════════════════════════════════════════════════════════════
STEP 13: Generate Predictions
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n🔮 Generating predictions on validation set...")

In [ ]:
# Predict all validation data (fast batch prediction)
y_pred_proba = model.predict(val_batches, verbose=1)
y_pred = (y_pred_proba > 0.5).astype("int32").flatten()

In [ ]:
# Get true labels
y_true = np.concatenate([labels.numpy() for _, labels in val_batches])

In [ ]:
print(f"✅ Predictions generated for {len(y_true)} samples")

═══════════════════════════════════════════════════════════════════════════
STEP 14: Confusion Matrix
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n📊 Creating confusion matrix...")

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Cat', 'Dog'],
    yticklabels=['Cat', 'Dog'],
    cbar_kws={'label': 'Count'},
    annot_kws={'size': 16, 'weight': 'bold'}
)
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('VGG16 Confusion Matrix', fontsize=14, fontweight='bold')

In [ ]:
# Add accuracy to plot
accuracy = accuracy_score(y_true, y_pred)
plt.text(
    0.5, -0.15,
    f'Overall Accuracy: {accuracy:.2%}',
    ha='center',
    transform=plt.gca().transAxes,
    fontsize=12,
    fontweight='bold'
)

In [ ]:
plt.tight_layout()
plt.savefig(f'plots/vgg16_confusion_matrix_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print(f"✅ Confusion matrix saved to: plots/vgg16_confusion_matrix_{timestamp}.png")

═══════════════════════════════════════════════════════════════════════════
STEP 15: Classification Report
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n" + "="*60)
print("  CLASSIFICATION REPORT")
print("="*60)

In [ ]:
# Generate classification report
report = classification_report(
    y_true,
    y_pred,
    target_names=['Cat', 'Dog'],
    digits=4
)

In [ ]:
print(report)

In [ ]:
# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()

In [ ]:
print("="*60)
print("  DETAILED METRICS")
print("="*60)
print(f"True Negatives (Cat→Cat):   {tn:,}")
print(f"False Positives (Cat→Dog):  {fp:,}")
print(f"False Negatives (Dog→Cat):  {fn:,}")
print(f"True Positives (Dog→Dog):   {tp:,}")
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy:.2%})")
print("="*60)

═══════════════════════════════════════════════════════════════════════════
STEP 16: Prediction Visualization
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n🖼️  Creating prediction visualization grid...")

In [ ]:
# Collect sample images
val_images_list = []
val_labels_list = []

In [ ]:
for images, labels in val_batches.take(5):
    val_images_list.append(images.numpy())
    val_labels_list.append(labels.numpy())

In [ ]:
# Stack all images and labels
val_images_array = np.vstack(val_images_list)
val_labels_array = np.concatenate(val_labels_list)

In [ ]:
# Predict on these samples
sample_preds = model.predict(val_images_array[:160], verbose=0)
sample_pred_classes = (sample_preds > 0.5).astype("int32").flatten()

In [ ]:
# Sample unique indices (no duplicates)
num_samples = 15
total_available = len(val_images_array[:160])
indices = random.sample(range(total_available), min(num_samples, total_available))

In [ ]:
# Create visualization grid
fig, axes = plt.subplots(3, 5, figsize=(15, 10))
axes = axes.ravel()

In [ ]:
for i, idx in enumerate(indices):
    if i >= 15:
        break
    
    # Get image, true label, and prediction
    img = val_images_array[idx]
    true_label = val_labels_array[idx]
    pred_label = sample_pred_classes[idx]
    pred_conf = sample_preds[idx][0]
    
    # Convert labels to names
    true_name = "Dog" if true_label == 1 else "Cat"
    pred_name = "Dog" if pred_label == 1 else "Cat"
    
    # Determine color (green if correct, red if wrong)
    color = 'green' if true_label == pred_label else 'red'
    
    # Plot image
    axes[i].imshow(img)
    axes[i].axis('off')
    
    # Add title with prediction
    title = f"True: {true_name}\nPred: {pred_name}"
    if true_label == pred_label:
        title += f"\n(Conf: {pred_conf:.2%})"
    else:
        title += f"\n(Conf: {pred_conf:.2%}) ❌"
    
    axes[i].set_title(title, color=color, fontweight='bold', fontsize=10)

In [ ]:
plt.suptitle(
    'VGG16: Sample Predictions (Green=Correct, Red=Wrong)',
    fontsize=16,
    fontweight='bold',
    y=1.02
)
plt.tight_layout()
plt.savefig(f'plots/vgg16_predictions_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print(f"✅ Prediction grid saved to: plots/vgg16_predictions_{timestamp}.png")

═══════════════════════════════════════════════════════════════════════════
STEP 17: Save Model & Metadata
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n💾 Saving final model and metadata...")

In [ ]:
# Save final model (in addition to best checkpoint)
final_model_path = f'models/vgg16_final_{timestamp}.h5'
model.save(final_model_path)

In [ ]:
# Create metadata
import json

In [ ]:
metadata = {
    'model_name': 'VGG16_Baseline',
    'timestamp': timestamp,
    'architecture': {
        'base_model': 'VGG16',
        'input_shape': [224, 224, 3],
        'trainable_params': int(np.sum([tf.size(w).numpy() for w in model.trainable_weights])),
        'total_params': int(np.sum([tf.size(w).numpy() for w in model.weights]))
    },
    'hyperparameters': {
        'batch_size': BATCH_SIZE,
        'epochs_trained': len(history.history['accuracy']),
        'max_epochs': EPOCHS,
        'learning_rate': float(LEARNING_RATE),
        'optimizer': 'Adam',
        'loss': 'binary_crossentropy'
    },
    'performance': {
        'final_train_accuracy': float(acc[-1]),
        'final_val_accuracy': float(val_acc[-1]),
        'best_val_accuracy': float(max(val_acc)),
        'final_train_loss': float(loss[-1]),
        'final_val_loss': float(val_loss[-1]),
        'best_val_loss': float(min(val_loss)),
        'overall_accuracy': float(accuracy),
        'precision': float(tp / (tp + fp)),
        'recall': float(tp / (tp + fn))
    },
    'dataset': {
        'name': 'cats_vs_dogs',
        'train_samples': int(ds_info.splits['train'].num_examples * 0.8),
        'val_samples': int(ds_info.splits['train'].num_examples * 0.2),
        'classes': ds_info.features['label'].names
    },
    'files': {
        'model': final_model_path,
        'training_log': f'logs/training_log_vgg16_{timestamp}.csv',
        'confusion_matrix': f'plots/vgg16_confusion_matrix_{timestamp}.png',
        'training_curves': f'plots/vgg16_training_curves_{timestamp}.png',
        'predictions': f'plots/vgg16_predictions_{timestamp}.png'
    }
}

In [ ]:
# Save metadata
metadata_path = f'models/metadata_vgg16_{timestamp}.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

In [ ]:
print(f"✅ Model saved to: {final_model_path}")
print(f"✅ Metadata saved to: {metadata_path}")

═══════════════════════════════════════════════════════════════════════════
STEP 18: Summary
═══════════════════════════════════════════════════════════════════════════

In [ ]:
print("\n" + "="*60)
print("  TRAINING SUMMARY")
print("="*60)
print(f"Model: VGG16 Baseline")
print(f"Epochs Trained: {len(history.history['accuracy'])}/{EPOCHS}")
print(f"Best Validation Accuracy: {max(val_acc):.2%}")
print(f"Final Test Accuracy: {accuracy:.2%}")
print(f"\nFiles Generated:")
print(f"  • Model: {final_model_path}")
print(f"  • Metadata: {metadata_path}")
print(f"  • Training curves: plots/vgg16_training_curves_{timestamp}.png")
print(f"  • Confusion matrix: plots/vgg16_confusion_matrix_{timestamp}.png")
print(f"  • Predictions: plots/vgg16_predictions_{timestamp}.png")
print(f"  • Training log: logs/training_log_vgg16_{timestamp}.csv")
print("="*60)
print("\n✅ VGG16 BASELINE TRAINING COMPLETE! 🐕🐈")
print("="*60)